# Midterm

## Options

In [ ]:
train_ModleA = False

## Imports

In [1]:
import numpy as np
from tensorflow import keras
import tensorflow as tf
import os
import datetime
import shutil
import re

logs_base_dir = "./logs"
shutil.rmtree(logs_base_dir, ignore_errors=True)
os.makedirs(logs_base_dir, exist_ok=True)

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu,True)
    except RuntimeError as e:
        print(e)

%load_ext tensorboard

2023-10-31 03:03:11.555292: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-31 03:03:11.602902: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-31 03:03:12.332441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-31 03:03:13.399683: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 0

## Step 1: Training a binary classifier

### Separating the data

In [2]:
label_list = []
cats = [0 for i in range(12500)]
dogs = [1 for i in range(12500)]
label_list.extend(cats)
label_list.extend(dogs)

In [3]:
ds = keras.utils.image_dataset_from_directory(
    directory='data/dogs_cats/train/',
    labels=label_list,
    label_mode='categorical',
    batch_size=256,
    image_size=(128,128),
    seed=0,
    validation_split=0.1,
    subset="both"
)

Found 25000 files belonging to 2 classes.
Using 22500 files for training.
Using 2500 files for validation.


2023-10-31 03:04:56.462306: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 03:04:56.462394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 03:04:56.462413: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 03:04:57.087500: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-10-31 03:04:57.087560: I tensorflow/compile

In [4]:
(train_ds, test_ds) = ds

### Preprocess and Augment the data

In [5]:
data_augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.1),
        keras.layers.RandomFlip("vertical"),
        keras.layers.RandomZoom(height_factor=0.1,width_factor=0.1)
    ]
)

### Perform Transfer Learning

### MobileNetV2

In [6]:
base_mobile_model = keras.applications.MobileNetV2(
    weights="imagenet",
    input_shape=(128,128,3),
    include_top=False
)

base_mobile_model.trainable = False

inputs = keras.Input(shape=(128,128,3))
x = data_augmentation(inputs)

scaled_layer = keras.layers.Rescaling(scale=1/255.0)
x = scaled_layer(x)

x = base_mobile_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(2)(x)
mobile_model = keras.Model(inputs, outputs)

mobile_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 mobilenetv2_1.00_128 (Func  (None, 4, 4, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 1280)              0     

In [7]:
keras.backend.clear_session()
mobile_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

epochs = 20
mobile_model.fit(train_ds, epochs=epochs, callbacks=[tensorboard_callback])

Epoch 1/20


2023-10-31 03:05:03.488528: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8801
2023-10-31 03:05:03.816200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-10-31 03:05:03.880253: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8d4d91ec30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-10-31 03:05:03.880289: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3080, Compute Capability 8.6
2023-10-31 03:05:03.885090: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-10-31 03:05:03.968896: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the p

88/88 [==============================] - 27s 247ms/step - loss: 0.4110 - categorical_accuracy: 0.8144
Epoch 2/20
88/88 [==============================] - 23s 240ms/step - loss: 0.2868 - categorical_accuracy: 0.8775
Epoch 3/20
88/88 [==============================] - 23s 242ms/step - loss: 0.2672 - categorical_accuracy: 0.8847
Epoch 4/20
88/88 [==============================] - 23s 241ms/step - loss: 0.2603 - categorical_accuracy: 0.8894
Epoch 5/20
88/88 [==============================] - 24s 245ms/step - loss: 0.2490 - categorical_accuracy: 0.8940
Epoch 6/20
88/88 [==============================] - 23s 243ms/step - loss: 0.2462 - categorical_accuracy: 0.8930
Epoch 7/20
88/88 [==============================] - 23s 242ms/step - loss: 0.2476 - categorical_accuracy: 0.8954
Epoch 8/20
88/88 [==============================] - 23s 242ms/step - loss: 0.2461 - categorical_accuracy: 0.8965
Epoch 9/20
88/88 [==============================] - 23s 241ms/step - loss: 0.2453 - categorical_accuracy: 0

#### Fine Tuning

In [8]:
keras.backend.clear_session()
base_mobile_model.trainable = True

mobile_model.compile(
    optimizer=keras.optimizers.Adam(1e-5),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[keras.metrics.CategoricalAccuracy()],
)

mobile_model.fit(train_ds, epochs=10, callbacks=[tensorboard_callback])

Epoch 1/10
88/88 [==============================] - 51s 303ms/step - loss: 0.2116 - categorical_accuracy: 0.9106
Epoch 2/10
88/88 [==============================] - 25s 254ms/step - loss: 0.1819 - categorical_accuracy: 0.9243
Epoch 3/10
88/88 [==============================] - 25s 259ms/step - loss: 0.1628 - categorical_accuracy: 0.9328
Epoch 4/10
88/88 [==============================] - 25s 257ms/step - loss: 0.1534 - categorical_accuracy: 0.9374
Epoch 5/10
88/88 [==============================] - 25s 256ms/step - loss: 0.1400 - categorical_accuracy: 0.9432
Epoch 6/10
88/88 [==============================] - 25s 258ms/step - loss: 0.1306 - categorical_accuracy: 0.9468
Epoch 7/10
88/88 [==============================] - 25s 260ms/step - loss: 0.1257 - categorical_accuracy: 0.9491
Epoch 8/10
88/88 [==============================] - 25s 259ms/step - loss: 0.1209 - categorical_accuracy: 0.9504
Epoch 9/10
88/88 [==============================] - 25s 259ms/step - loss: 0.1149 - categorical_

### Saving Model A

In [9]:
mobile_model.save_weights("models/modelA.ckpt")

In [10]:
mobile_model.load_weights('models/modelA.ckpt')